# Amex to Concur transaction matching

## Process

1. Download AmericanExpress transactions in Excel format from [AmericanExpress website](https://www.americanexpress.com/)
2. Run Concur report
   1. Launch Concur
   2. Click Reporting -> Intelligence
   3. In the Concur Reporting Home page click on the "Public Folder" tab if you are not on that tab
   4. Click "International Business Machines Corporation (p0012308gu3u)"
   5. Click "55 - Credit Card Admin"
   6. Click "FI0201 - Credit Card Transactions"
      1. Set date range
      2. Select all Payment Types
      3. Select all Transaction Statuses
      4. Search and "Insert" yourself to the "Choice" of Employees
   1. After the report is generated. Select the option from top right to get Excel 2007 Data format and save the resulting Excel.
    


In [ ]:
##############################################################
# Configuration values
##############################################################

import os

## Data Folder and File Names
DATA_DIR                = './data'

# CONCUR_TRANS_FILE_PATH  = os.path.join(DATA_DIR, 'FI0201 - Credit Card Transactions.xlsx')
# AMEX_TRANS_FILE_PATH    = os.path.join(DATA_DIR, 'amex-transaction.xlsx')

CONCUR_TRANS_FILE_PATH  = os.path.join(DATA_DIR, 'concur-2020.xlsx')
AMEX_TRANS_FILE_PATH    = os.path.join(DATA_DIR, 'amex-2020.xlsx')

REPORT_FILE_PATH        = os.path.join(DATA_DIR, 'amex-transaction-match-report-2020.xlsx')

## Vendor name lenghth use to match
VENDOR_GROUP_BY_LENGTH  = 16

In [ ]:
import numpy as np
import pandas as pd
import re

import inputfileloader as ifl

## Load Concur Transactions

1. Load the Concur Transaction
1. Cluster the concur transactions by DescValue and transaction date. This process clusters multiple transactions in one calendar day into one to simplify the logic of matching multiple charges from one vendor with same amount.


In [ ]:
# load concur report Excel file
concur_tran = ifl.load_concur_report(CONCUR_TRANS_FILE_PATH, 'Merchant', VENDOR_GROUP_BY_LENGTH)

# # Select only Corporate Card transaction (no cash...)
# concurreport = concurreport[ concurreport['Payment Type'] == 'US American Express Corporate Card' ]

# Cluster concur transactions by vendor and dates
concur_cluster = ifl.cluster_tran_entries(
    concur_tran,
    'DescValue',
    'Transaction Date',
    'Posted Amount (credit card currency)',
    '1 hour')

concur_cluster.columns = ['Amount_sum_c', 'Amount_count_c', 'Date_min_c', 'Date_max_c']

concur_cluster['amex_group_num'] = np.NaN


## Load Amex Transactions

1. Load the Amex transactions
1. Group the expense by description's first 16 characters and transaction date. This process clusters multiple transactions in one calendar day into one to simplify the logic of matching multiple charges from one vendor with same amount.


In [ ]:

amextran = ifl.load_amex_tran(AMEX_TRANS_FILE_PATH, 'Description', VENDOR_GROUP_BY_LENGTH)

# Cluster Amex transactions by description and date
amex_cluster = ifl.cluster_tran_entries(
    amextran,
    'DescValue',
    'Date',
    'Amount',
    '1 hour')

amex_cluster.columns = ['Amount_sum', 'Amount_count', 'Date_min', 'Date_max']


### First Match 

Match Amex transaction clusters with Concur transaction clusters with same description and close in date

In [ ]:

def match_amex_cluster (amex_cluster_row):
    
    desc_value = amex_cluster_row.name[0]
    amex_group_num = amex_cluster_row.name[1]
    amount = amex_cluster_row['Amount_sum']
    count = amex_cluster_row['Amount_count']
    date_min = amex_cluster_row['Date_min']
    date_max = amex_cluster_row['Date_max']


    # Filter concur report with DescValue
    # concur_match_desc = concur_cluster[ concur_cluster.index.str.startswith(desc_value) ]
    try:
        concur_match_desc = concur_cluster.loc[desc_value]
    except KeyError:
        concur_match_desc = None

    # print('concur cluster with same desc: ', concur_match_desc)

    concur_match = [None] * 6


    if (concur_match_desc is not None and concur_match_desc.empty == False):
        concur_match_desc['min_max_time_diff'] = (date_min - concur_match_desc['Date_max_c']).abs()
        concur_match_desc['min_min_time_diff'] = (date_min - concur_match_desc['Date_min_c']).abs()
        concur_match_desc['max_min_time_diff'] = (date_max - concur_match_desc['Date_min_c']).abs()
        concur_match_desc['max_max_time_diff'] = (date_max - concur_match_desc['Date_max_c']).abs()

        concur_match_desc['min_diff'] = concur_match_desc.loc[:, ['min_min_time_diff', 'min_max_time_diff', 'max_min_time_diff', 'max_max_time_diff']].min(axis=1)

        concur_match_desc['concur_amex_fraction'] = concur_match_desc['Amount_sum_c'] / amount
            
        concur_match_desc = concur_match_desc[ (concur_match_desc['concur_amex_fraction'] > 0.8) &
            (concur_match_desc['concur_amex_fraction'] < 1.2) ]
        

        # if concur_match_desc is not None :
        #     print ('concur_match_desc for {} | {} found {} concur record'.format(desc_value, amex_group_num, len(concur_match_desc.index)))

        min_date_diff = concur_match_desc['min_diff'].min()

        if min_date_diff < pd.to_timedelta('6 days') :

            # print (concur_match_desc['min_diff'].idxmin())
            
            concurrow = concur_match_desc[ concur_match_desc.index == concur_match_desc['min_diff'].idxmin() ].iloc[0]
            # print('concur condidates: ', concur_match_desc)
            # print('match concur cluster: ', concurrow)
            # concurrow = concur_match_desc[ concur_match_desc['min_diff'].idxmin()]

            concur_group_num = concur_match_desc['min_diff'].idxmin()

            concur_match = [ 
                concur_group_num,
                concurrow['Amount_sum_c'], 
                concurrow['Amount_count_c'], 
                concurrow['Date_min_c'], 
                concurrow['Date_max_c'],
                concurrow['min_diff']]
            
            # print(concur_match_desc['min_diff'].idxmin())
            concur_cluster.loc[(desc_value, concur_group_num), 'amex_group_num'] = amex_group_num

            # concur_cluster.iloc[concur_match_desc['min_diff'].idxmin(), 4] = amex_group_num
        # print(concur_match_desc['min_diff'].min(), concur_match_desc['min_diff'].idxmin())
        # print(concur_match_desc['min_diff'].min())

    matchresult = [desc_value, amex_group_num, amount, count, date_min, date_max] + concur_match

    # print('Match Result: ', matchresult)

    return pd.Series(matchresult, index=['desc', 'amex_group_num', 'amex_sum', 'amex_count', 'amex_cluster_start', 'amex_cluster_end', 'concur_group_num', 'concur_sum', 'concur_count', 'concur_start', 'concur_end', 'min_date_diff'])


amex_cluster_match = amex_cluster.apply(match_amex_cluster, axis=1, result_type='expand')

amex_cluster_match = amex_cluster_match.reindex()

amex_cluster_paid = amex_cluster_match[ (amex_cluster_match['amex_sum'] - amex_cluster_match['concur_sum'] < 0.01) ]

amex_cluster_mismatch = amex_cluster_match[ (amex_cluster_match['amex_sum'] - amex_cluster_match['concur_sum'] > 0.01) ]




## Second Match

Match Amex transaction clusters to Concur transaction clusters by just amount and date range to match the ones with different description between Amex and Concur.


In [ ]:


def guess_amex_cluster(amex_no_match_row, amount_tolerance, date_tolerance):

    amex_amount = amex_no_match_row['amex_sum']
    amex_start = amex_no_match_row['amex_cluster_start']
    amex_end = amex_no_match_row['amex_cluster_end']
    amex_group_num = amex_no_match_row.name[1]

    possible_match = concur_cluster[ 
        ( (1 + amount_tolerance) > (concur_cluster['Amount_sum_c'] / amex_amount) )
        & ( (concur_cluster['Amount_sum_c'] / amex_amount) > (1 - amount_tolerance) )
        & ( amex_end + date_tolerance > concur_cluster['Date_max_c'] )  
        & ( concur_cluster['Date_min_c'] >  amex_start - date_tolerance ) 
        & (concur_cluster['amex_group_num'].isnull()) ]

    # print ('possible_match for {} from {} to {}:'.format(amex_amount, amex_start, amex_end), possible_match)

    if (possible_match is None or possible_match.empty):
        amex_no_match_row['concur_desc'] = None
    else:
        a_guess = possible_match.iloc[0]
        amex_no_match_row['concur_group_num'] = a_guess.name[1]
        amex_no_match_row['concur_sum'] = a_guess['Amount_sum_c']
        amex_no_match_row['concur_count'] = a_guess['Amount_count_c']
        amex_no_match_row['concur_start'] = a_guess['Date_min_c']
        amex_no_match_row['concur_end'] = a_guess['Date_max_c']
        amex_no_match_row['concur_desc'] = a_guess.name[0]

        concur_cluster.loc[a_guess.name, 'amex_group_num'] = amex_group_num

    return pd.Series(amex_no_match_row, index=['desc', 'amex_group_num', 'amex_sum', 'amex_count', 'amex_cluster_start', 'amex_cluster_end', 'concur_group_num', 'concur_sum', 'concur_count', 'concur_start', 'concur_end', 'min_date_diff', 'concur_desc'])

amex_cluster_no_match = amex_cluster_match [ (amex_cluster_match['concur_sum'].isnull()) ]

# print (concur_cluster['amex_group_num'])
amex_cluster_guess_match = amex_cluster_no_match.apply(
    guess_amex_cluster, 
    args = (0.01, pd.to_timedelta('6 days')),
    axis = 1, 
    result_type = 'expand',
    )



amex_guess_no_match = amex_cluster_no_match [ (amex_cluster_no_match['concur_sum'].isnull()) ]

# amex_guess2 = amex_guess_no_match.apply(
#     guess_amex_cluster, 
#     args = (0.08, pd.to_timedelta('6 days')),
#     axis = 1, 
#     result_type = 'expand', 
#     )

# amex_guess2

## Write out results

Save the results to an Excel workbook.


In [ ]:
output_writer = pd.ExcelWriter(REPORT_FILE_PATH)

amextran.to_excel(output_writer, sheet_name='amex trans')
amex_cluster.to_excel(output_writer, sheet_name='amex cluster')

concur_tran.to_excel(output_writer, sheet_name='concur trans')
concur_cluster.to_excel(output_writer, sheet_name='concur cluster')

amex_cluster_match.to_excel(output_writer, sheet_name='amex match')
amex_cluster_paid.to_excel(output_writer, sheet_name='amex paid full')

if amex_cluster_mismatch.empty == False:
    amex_cluster_mismatch.to_excel(output_writer, sheet_name='amex mismatch')
# amex_cluster_no_match.to_excel(output_writer, sheet_name='amex no match')
amex_cluster_guess_match.to_excel(output_writer, sheet_name='amex guess 1')
# amex_guess2.to_excel(output_writer, sheet_name='amex guess 2')

output_writer.save()
output_writer.close()